In [2]:
from datasets import load_dataset
import sys
sys.path.insert(0,'/content/')

from vector_databases.chroma_instance import ChromaInstance
from models.encoder import Encoder
from models.llm import GenerativeModel

### Data

In [3]:
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"

data = load_dataset(dataset_name).get('train')

corpus = [x for x in data['context'] if x != ""]
questions = [x for x in data['instruction'] if x != ""]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
corpus[:5]

["Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches unti

In [5]:
questions[:5]

['When did Virgin Australia start operating?',
 'Which is a species of fish? Tope or Rope',
 'Why can camels survive for long without water?',
 "Alice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?",
 'When was Tomoaki Komorida born?']

### Models setup

In [6]:
encoder = Encoder('all-distilroberta-v1')
generative_llm = GenerativeModel('tiiuae/falcon-7b')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Database setup

In [7]:
vector_db = ChromaInstance()
vector_db.create_index('rag_demo_collection')

### Text vectorization and indexing

In [8]:
vectorized_text = encoder.encode(corpus, bsize=64)
vector_db.add_to_collection(vectorized_text, corpus)

Batches:   0%|          | 0/70 [00:00<?, ?it/s]

### Retrieval step

In [9]:
question = questions[11223]
question

'What are some board games that involve strategy?'

In [10]:
vectorized_question = encoder.encode(question)
retrieved_document = vector_db.search(vectorized_question)[0]
retrieved_document

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['Chess is an abstract strategy game that involves no hidden information and no use of dice or cards. It is played on a chessboard with 64 squares arranged in an eight-by-eight grid. At the start, each player controls sixteen pieces: one king, one queen, two rooks, two bishops, two knights, and eight pawns. White moves first, followed by Black. Checkmating the opponent\'s king involves putting the king under immediate attack (in "check") whereby there is no way for it to escape. There are also several ways a game can end in a draw.']

Generative Step

In [13]:
generative_llm.generate_answer(question, retrieved_document, max_length=512)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'Result: Context: \'\'\'[\'Chess is an abstract strategy game that involves no hidden information and no use of dice or cards. It is played on a chessboard with 64 squares arranged in an eight-by-eight grid. At the start, each player controls sixteen pieces: one king, one queen, two rooks, two bishops, two knights, and eight pawns. White moves first, followed by Black. Checkmating the opponent\\\'s king involves putting the king under immediate attack (in "check") whereby there is no way for it to escape. There are also several ways a game can end in a draw.\']\'\'\' Question: \'\'\'What are some board games that involve strategy?\'\'\'             answer the Question according to the Context. I got **\'Risk\'**, but is there any other board game which involves strategy? Monopoly is another game that I think would fit your question.\n\nMonopoly is a commercial board game based on land ownership and property acquisition. Risk or monopoly are classic I guess.  Not what I really wanted to